In [1]:
import pandas as pd
import numpy as np

In [2]:
from utils.freebase import *
from utils.data_initial import *
from utils.data_generated import *

# Ethnicity values

In [3]:
df = load_cmu_character_metadata()

In [4]:
etnicities = list(df.actor_ethnicity.dropna().unique())

In [5]:
mappings = {}
not_found = []
for etn in etnicities:
    val = query_freebase_id_from_wikidata(etn)
    if val == None:
        not_found.append(etn)
    else:
        mappings[etn] = val

weird result for /m/019kn7
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q161652'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Japanese people'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31340083'}, 'sLabel': {'type': 'literal', 'value': 'Q31340083'}}]

weird result for /m/0j6x8
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q170355'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Indigenous Australians'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q12060728'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aboriginal Australians'}}]

weird result for /m/062_25
[{'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1065371'}, 'sLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Italian Brazilians'}}, {'s': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q54864438'}, 'sLabel': {'type': 'literal', 'value': 'Q54864438'}}]

weird result for /m/0640_7

In [6]:
# manually correct the mappings that were under an unexpected format
mappings['/m/019kn7'] = 'Japanese'
mappings['/m/0j6x8'] = 'Aboriginal Australians'
mappings['/m/062_25'] = 'Italian Brazilians'
mappings['/m/0640_7q'] = 'Moroccan Jews'
mappings['/m/0180zw'] = 'Kikuyu'
mappings['/m/09snp5'] = 'Muhajir'
mappings['/m/03x1x'] = 'Haudenosaunee Confederacy'
mappings['/m/06bkf'] = 'Quebeckers'

In [7]:
# create dataframe
# set mappings for ids without values to NaN
mappings.update({id: np.NaN for id in not_found})

df = pd.DataFrame(data=mappings.items(), index=range(0,len(mappings)), columns=['freebase_id', 'ethnicity'])

In [8]:
# save dataframe
df.to_pickle(PATH_DATA_GEN + FILENAME_ETHNICITIES)